# Data Cleaning

In [1]:
import pandas as pd
import numpy as np
print('Import sucessfully')

Import sucessfully


In [2]:
import os
os.chdir(r'C:\Users\nguye\OneDrive\Máy tính\New folder (2)\Momo\Data')

os.getcwd()    # kiểm tra
os.listdir()   # phải thấy 4 file csv


['Cashback.xlsx',
 'Commission.xlsx',
 'MoMo Talent 2024_DA_Case Study Round_Questions.xlsx',
 'Transactions.xlsx',
 'User_Info.xlsx']

In [3]:
#Load xlsx files into python
users = pd.read_excel('User_Info.xlsx')
transactions = pd.read_excel('Transactions.xlsx')
commission = pd.read_excel('Commission.xlsx')
cashback = pd.read_excel('Cashback.xlsx')


## Cleaning the users table

In [4]:
#Cleaning the users dataframe
users.head()

,User_id,First_tran_date,Location,Age,Gender
0,41654498,2018-05-02,HCMC,33_to_37,MALE
1,51276281,2019-12-08,Other Cities,unknown,MALE
2,49152375,2019-12-20,Other Cities,23_to_27,MALE
3,5971050,2019-08-01,HCMC,28_to_32,MALE
4,48134464,2019-08-23,Other Cities,18_to_22,FEMALE


In [5]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13428 entries, 0 to 13427
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User_id          13428 non-null  int64 
 1   First_tran_date  13428 non-null  object
 2   Location         13428 non-null  object
 3   Age              13428 non-null  object
 4   Gender           13428 non-null  object
dtypes: int64(1), object(4)
memory usage: 524.7+ KB


In [6]:
#Transform unknown value into NaN
users_cleaned = users.replace(['unknown', 'Unknown'],np.nan)
users_cleaned = pd.DataFrame(users_cleaned)
users_cleaned

,User_id,First_tran_date,Location,Age,Gender
0,41654498,2018-05-02,HCMC,33_to_37,MALE
1,51276281,2019-12-08,Other Cities,NaN,MALE
2,49152375,2019-12-20,Other Cities,23_to_27,MALE
3,5971050,2019-08-01,HCMC,28_to_32,MALE
4,48134464,2019-08-23,Other Cities,18_to_22,FEMALE
...,...,...,...,...,...
13423,39810630,2018-03-19,Other Cities,28_to_32,Nam
13424,47628375,2019-09-05,HCMC,28_to_32,female
13425,46334970,2019-04-23,Other Cities,NaN,male
13426,50760533,2019-11-13,Other Cities,18_to_22,Nữ


In [7]:
#Checking the number of user_id and distinct user_id
users_cleaned['User_id'].count() --13428

26856

In [8]:
len(users_cleaned['User_id'].unique()) --13390

26780

In [9]:
users_cleaned[users_cleaned['User_id'].duplicated(keep=False)].sort_values(by='User_id')

,User_id,First_tran_date,Location,Age,Gender
12118,1763134,2020-08-10,HCMC,28_to_32,Nữ
3456,1763134,2020-08-10,HCMC,33_to_37,MALE
12772,3417428,2019-08-15,HCMC,33_to_37,female
2817,3417428,2019-08-15,HCMC,NaN,FEMALE
5164,3945743,2019-11-06,HN,28_to_32,MALE
...,...,...,...,...,...
125,55789348,2020-05-06,HCMC,23_to_27,MALE
5572,56354395,2020-06-09,Other Cities,>37,FEMALE
919,56354395,2020-06-09,NaN,NaN,FEMALE
11628,58928502,2020-09-27,Other Cities,28_to_32,Nữ


In [10]:
# Drop duplicated user_id rows, retain only the first user_id appears
users_cleaned = users_cleaned.drop_duplicates(subset=['User_id'], keep='first')
# Reset the index
users_cleaned.reset_index(drop=True, inplace=True)
# Display the first few rows to check
users_cleaned.head()

,User_id,First_tran_date,Location,Age,Gender
0,41654498,2018-05-02,HCMC,33_to_37,MALE
1,51276281,2019-12-08,Other Cities,NaN,MALE
2,49152375,2019-12-20,Other Cities,23_to_27,MALE
3,5971050,2019-08-01,HCMC,28_to_32,MALE
4,48134464,2019-08-23,Other Cities,18_to_22,FEMALE


In [11]:
#Count the number of missing values in each columns
users_missing = users_cleaned.isna()
for column in users_missing.columns.values.tolist():
    print (users_missing[column].value_counts())

User_id
False    13390
Name: count, dtype: int64
First_tran_date
False    13390
Name: count, dtype: int64
Location
False    12631
True       759
Name: count, dtype: int64
Age
False    10873
True      2517
Name: count, dtype: int64
Gender
False    13390
Name: count, dtype: int64


In [12]:
#Checking data of Gender column
users_cleaned['Gender'].unique()

array(['MALE', 'FEMALE', 'Nữ', 'M', 'female', 'Nam', 'f', 'male'],
      dtype=object)

Only columns `Location` and `Age` have null values

In [13]:
users_cleaned['Age'] = users_cleaned['Age'].fillna('Unknown')

C:\Users\nguye\AppData\Local\Temp\ipykernel_20180\318435671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_cleaned['Age'] = users_cleaned['Age'].fillna('Unknown')


In [14]:
#Fixing data intergiry in column Gender
users_cleaned['Gender'] = (
    users_cleaned['Gender']
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({
        'male': 'MALE',
        'm': 'MALE',
        'nam': 'MALE',
        'female': 'FEMALE',
        'f': 'FEMALE',
        'nữ': 'FEMALE',
        'nu': 'FEMALE'
    })
)


C:\Users\nguye\AppData\Local\Temp\ipykernel_20180\2686808466.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_cleaned['Gender'] = (


In [15]:
users_cleaned['Gender'].value_counts()


Gender
MALE      7796
FEMALE    5594
Name: count, dtype: int64

In [16]:
#Checking data of Location column
users_cleaned['Location'].unique()

array(['HCMC', 'Other Cities', 'Other', 'HN', nan, 'Ho Chi Minh City'],
      dtype=object)

In [17]:
users_cleaned['Location'] = (
    users_cleaned['Location']
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({
        'hcmc': 'HCMC',
        'ho chi minh city': 'HCMC',
        'hn': 'HN',
        'hanoi': 'HN',
        'other': 'Other',
        'other cities': 'Other',
        'nan': 'Other'
    })
)

C:\Users\nguye\AppData\Local\Temp\ipykernel_20180\2451106523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_cleaned['Location'] = (


In [18]:
users_cleaned['Location'].value_counts()


Location
Other    7794
HCMC     4162
HN       1434
Name: count, dtype: int64

In [19]:
#Checking the logic of first_tran_date df
users_cleaned['First_tran_date'].sort_values(ascending=False).head(75)

7100     9920-12-16
8773     9920-12-12
10166    9920-12-06
7399     9920-05-31
6905     9920-04-27
            ...    
1374     9917-09-23
4075     9917-09-22
6697     3020-12-01
6025     3020-05-28
5150     2020-12-31
Name: First_tran_date, Length: 75, dtype: object

There are values in first_tran_date column with the year greater than 2020. Because the dataset only contain values from before the year 2020, all the year must be below 2020. And i see a pattern that the first two number '20' of the year is converted into '99' and ''30'.

In [20]:
# Define a function to correct the year
def correct_year(date_str):
    if pd.notnull(date_str):  # Check if the date string is not null
        parts = date_str.split('-')
        if len(parts) == 3 and parts[0].startswith(('99','30')): #len(parts)==3 ensure parts includes 3 part YYYY,MM,DD
            parts[0] = '20' + parts[0][2:]  # Change '99' and '30' to '20'
            return '-'.join(parts)
    return date_str

# Apply the function to the first_tran_date column
users_cleaned['First_tran_date'] = users_cleaned['First_tran_date'].apply(correct_year)

C:\Users\nguye\AppData\Local\Temp\ipykernel_20180\3047895604.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_cleaned['First_tran_date'] = users_cleaned['First_tran_date'].apply(correct_year)


In [21]:
users_cleaned['First_tran_date'].sort_values(ascending=False).head(50)

5150     2020-12-31
3051     2020-12-31
6228     2020-12-31
115      2020-12-31
2026     2020-12-31
10236    2020-12-31
9550     2020-12-30
6550     2020-12-30
4992     2020-12-30
5525     2020-12-29
12035    2020-12-27
2327     2020-12-27
12247    2020-12-27
2471     2020-12-27
5907     2020-12-25
3086     2020-12-25
1692     2020-12-24
13220    2020-12-23
2994     2020-12-23
11822    2020-12-22
3253     2020-12-22
8694     2020-12-21
3112     2020-12-21
13060    2020-12-20
11109    2020-12-20
5506     2020-12-19
10346    2020-12-19
2197     2020-12-18
1971     2020-12-18
3588     2020-12-18
4811     2020-12-17
2458     2020-12-17
8513     2020-12-17
6373     2020-12-16
12050    2020-12-16
6594     2020-12-16
7100     2020-12-16
1421     2020-12-16
6392     2020-12-15
196      2020-12-14
12476    2020-12-13
547      2020-12-13
1065     2020-12-12
3968     2020-12-12
8773     2020-12-12
11521    2020-12-12
10275    2020-12-11
6146     2020-12-11
3692     2020-12-11
10786    2020-12-11


In [22]:
#Changing the data type
users_cleaned = users_cleaned[['User_id','First_tran_date','Location','Age','Gender']]
users_cleaned.info()
users_cleaned['First_tran_date'] = users_cleaned['First_tran_date'].astype('datetime64[ns]')
users_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13390 entries, 0 to 13389
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User_id          13390 non-null  int64 
 1   First_tran_date  13390 non-null  object
 2   Location         13390 non-null  object
 3   Age              13390 non-null  object
 4   Gender           13390 non-null  object
dtypes: int64(1), object(4)
memory usage: 523.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13390 entries, 0 to 13389
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   User_id          13390 non-null  int64         
 1   First_tran_date  13390 non-null  datetime64[ns]
 2   Location         13390 non-null  object        
 3   Age              13390 non-null  object        
 4   Gender           13390 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)

In [23]:
#Final users data
users_cleaned.head(10)

,User_id,First_tran_date,Location,Age,Gender
0,41654498,2018-05-02,HCMC,33_to_37,MALE
1,51276281,2019-12-08,Other,Unknown,MALE
2,49152375,2019-12-20,Other,23_to_27,MALE
3,5971050,2019-08-01,HCMC,28_to_32,MALE
4,48134464,2019-08-23,Other,18_to_22,FEMALE
5,56702663,2020-08-08,Other,Unknown,MALE
6,53564527,2020-02-16,Other,Unknown,MALE
7,40659574,2017-12-28,Other,28_to_32,MALE
8,7571525,2018-04-20,HCMC,33_to_37,MALE
9,28716109,2019-12-17,Other,33_to_37,FEMALE


Table `users` is cleaned

## Cleaning the transactions table

In [24]:
transactions_cleaned = transactions.replace(['unknown', 'Unknown'],np.nan)
transactions_cleaned = pd.DataFrame(transactions_cleaned)
transactions_cleaned.head(20)

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status
0,21269588,4169517626,2020-01-01,"10,000",13,NaN
1,28097592,4170276686,2020-01-01,"20,000",13,NaN
2,47435144,4166729310,2020-01-01,"10,000",12,NaN
3,29080935,4174460303,2020-01-01,"10,000",13,NaN
4,14591075,4168216749,2020-01-01,"10,000",12,NaN
5,43155015,4171808289,2020-01-01,"10,000",12,NaN
6,50568745,4168401164,2020-01-01,"10,000",13,NaN
7,38802645,4167009272,2020-01-01,"20,000",12,NaN
8,28698657,4166562735,2020-01-01,"10,000",15,NaN
9,42557469,4168300611,2020-01-01,"50,000",12,NaN


In [25]:
transactions_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          13495 non-null  int64 
 1   order_id         13495 non-null  int64 
 2   Date             13495 non-null  object
 3   Amount           13495 non-null  object
 4   Merchant_id      13495 non-null  int64 
 5   Purchase_status  2235 non-null   object
dtypes: int64(3), object(3)
memory usage: 632.7+ KB


In [26]:
#Checking the number order_id and distinct order_id
len(transactions_cleaned) --13495

26990

In [27]:
len(transactions_cleaned['order_id'].unique()) --13495

26990

All the order_id in `transaction` table are unique

In [28]:
#Checking missing data
transactions_missing = transactions_cleaned.isna()
for column in transactions_missing.columns.values.tolist():
    print (transactions_missing[column].value_counts())

transactions_cleaned['Purchase_status'].value_counts()


user_id
False    13495
Name: count, dtype: int64
order_id
False    13495
Name: count, dtype: int64
Date
False    13495
Name: count, dtype: int64
Amount
False    13495
Name: count, dtype: int64
Merchant_id
False    13495
Name: count, dtype: int64
Purchase_status
True     11260
False     2235
Name: count, dtype: int64


Purchase_status
Mua hộ    2235
Name: count, dtype: int64

Only column `Purchase_status` have missing values

In [29]:
transactions_cleaned['Purchase_status'] = (
    transactions_cleaned['Purchase_status']
    .eq('Mua hộ')
    .astype(int)
)

transactions_cleaned['Purchase_status'].value_counts()

Purchase_status
0    11260
1     2235
Name: count, dtype: int64

In [30]:
transactions_cleaned['Merchant_id'].value_counts()

Merchant_id
12    6777
13    3783
14    2321
15     611
16       3
Name: count, dtype: int64

In [31]:
transactions_cleaned['Date'].sort_values(ascending=False).head()

9812    30/9/2020
9763    30/9/2020
9778    30/9/2020
9777    30/9/2020
9776    30/9/2020
Name: Date, dtype: object

There is no date format phenomenon as users table

In [32]:
#Changing data type
transactions_cleaned['Date'] = transactions_cleaned['Date'].astype('datetime64[ns]')
transactions_cleaned['Amount'] = transactions_cleaned['Amount'].str.replace(',', '')
transactions_cleaned['Amount'] = transactions_cleaned['Amount'].astype('int64')
transactions_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13495 entries, 0 to 13494
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   user_id          13495 non-null  int64         
 1   order_id         13495 non-null  int64         
 2   Date             13495 non-null  datetime64[ns]
 3   Amount           13495 non-null  int64         
 4   Merchant_id      13495 non-null  int64         
 5   Purchase_status  13495 non-null  int32         
dtypes: datetime64[ns](1), int32(1), int64(4)
memory usage: 580.0 KB


In [33]:
transactions_cleaned.head()

,user_id,order_id,Date,Amount,Merchant_id,Purchase_status
0,21269588,4169517626,2020-01-01,10000,13,0
1,28097592,4170276686,2020-01-01,20000,13,0
2,47435144,4166729310,2020-01-01,10000,12,0
3,29080935,4174460303,2020-01-01,10000,13,0
4,14591075,4168216749,2020-01-01,10000,12,0


Table `transactions` is cleaned

## Cleaning the commission table

In [34]:
commission


,Merchant_name,Merchant_id,Rate_pct
0,Viettel,12,2
1,Mobifone,13,3
2,Vinaphone,14,4
3,Vietnamobile,15,4
4,Gmobile,16,4


Table `commission` is cleaned

## Cleaning the cashback table

In [35]:
# Tạo bảng mới cashback_cleaned
cashback_cleaned = cashback.copy()

# Gộp cột current
cashback_cleaned['cashback_current'] = (
    cashback_cleaned['% cashback (current)']
    .fillna(cashback_cleaned['Unnamed: 2'])
)

# Gộp cột proposal
cashback_cleaned['cashback_proposal'] = (
    cashback_cleaned['% cashback (proposal)']
    .fillna(cashback_cleaned['Unnamed: 4'])
)

# Chỉ giữ các cột cần thiết
cashback_cleaned = cashback_cleaned[
    ['Merchant', 'cashback_current', 'cashback_proposal']
]

cashback_cleaned


,Merchant,cashback_current,cashback_proposal
0,Viettel,1.0,2.0
1,Mobifone,1.0,2.5
2,Vinaphone,1.0,3.0
3,Vietnamobile,1.0,3.0
4,Gmobile,1.0,3.0


## Merge Commission + Cashback

In [36]:
# Copy
commission_df = commission.copy()
cashback_df = cashback_cleaned.copy()

# Chuẩn hoá merchant
commission_df['Merchant_name'] = commission_df['Merchant_name'].str.strip().str.lower()
cashback_df['Merchant'] = cashback_df['Merchant'].str.strip().str.lower()

# LEFT JOIN + drop cột thừa
commission_cashback = commission_df.merge(
    cashback_df[['Merchant', 'cashback_current', 'cashback_proposal']],
    left_on='Merchant_name',
    right_on='Merchant',
    how='left'
).drop(columns=['Merchant'])

# Đổi tên cột
commission_cashback = commission_cashback.rename(columns={
    'Merchant_name': 'Merchant',
    'Rate_pct': 'commission_rate'
})

# Chuẩn hoá % → decimal
commission_cashback[['commission_rate',
                      'cashback_current',
                      'cashback_proposal']] /= 100

# Tính margin
commission_cashback['net_margin_current'] = (
    commission_cashback['commission_rate']
    - commission_cashback['cashback_current']
)

commission_cashback['net_margin_proposal'] = (
    commission_cashback['commission_rate']
    - commission_cashback['cashback_proposal']
)

# Sắp xếp cột
commission_cashback = commission_cashback[
    [
        'Merchant',
        'Merchant_id',
        'commission_rate',
        'cashback_current',
        'net_margin_current',
        'cashback_proposal',
        'net_margin_proposal'
    ]
]

commission_cashback


,Merchant,Merchant_id,commission_rate,cashback_current,net_margin_current,cashback_proposal,net_margin_proposal
0,viettel,12,0.02,0.01,0.01,0.020,0.000
1,mobifone,13,0.03,0.01,0.02,0.025,0.005
2,vinaphone,14,0.04,0.01,0.03,0.030,0.010
3,vietnamobile,15,0.04,0.01,0.03,0.030,0.010
4,gmobile,16,0.04,0.01,0.03,0.030,0.010


# Import data in MS SQL SERVER

In [37]:
import pyodbc
from sqlalchemy import create_engine

server = r'.\SQLEXPRESS'
database = 'Momo_project'

connection_string = (
    "mssql+pyodbc://@"
    f"{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

engine = create_engine(connection_string)


In [38]:
#Import three dataframe into three tables
# Create and insert data into 'users' table
users_cleaned.to_sql('users', con=engine, if_exists='replace', index=False)

# Create and insert data into 'transactions' table
transactions_cleaned.to_sql('transactions', con=engine, if_exists='replace', index=False)

# Create and insert data into 'commission' table
commission.to_sql('commission', con=engine, if_exists='replace', index=False)

# Create and insert data into 'cashback' table
cashback_cleaned.to_sql('cashback', con=engine, if_exists='replace', index=False)

# Create and insert data into 'commission_cashback' table
commission_cashback.to_sql('commission_cashback', con=engine, if_exists='replace', index=False)

print("DataFrames have been successfully imported into the SQL Server database.")

c:\Users\nguye\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:1636: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())


DataFrames have been successfully imported into the SQL Server database.


In [39]:
import pyodbc as odbc

server = r'.\SQLEXPRESS'
database = 'Momo_project'

cnxn = odbc.connect(
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=.\SQLEXPRESS;'
    r'DATABASE=Momo_project;'
    r'Trusted_Connection=yes;'
)

print(cnxn)
cursor_cnxn = cnxn.cursor()


In [40]:
#!pip install ipython-sql


In [41]:
# Load the sql extension for SQL magic
%load_ext sql
# Connect to the database using the connection string
%sql $connection_string

c:\Users\nguye\AppData\Local\Programs\Python\Python310\lib\site-packages\sql\connection.py:55: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  self.internal_connection = engine.connect()


In [42]:
#!pip install prettytable==3.9.0


In [43]:
%%sql
SELECT TOP 10 * FROM users;

 * mssql+pyodbc://@.\SQLEXPRESS/Momo_project?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes
Done.


User_id,First_tran_date,Location,Age,Gender
41654498,2018-05-02 00:00:00,HCMC,33_to_37,MALE
51276281,2019-12-08 00:00:00,Other,Unknown,MALE
49152375,2019-12-20 00:00:00,Other,23_to_27,MALE
5971050,2019-08-01 00:00:00,HCMC,28_to_32,MALE
48134464,2019-08-23 00:00:00,Other,18_to_22,FEMALE
56702663,2020-08-08 00:00:00,Other,Unknown,MALE
53564527,2020-02-16 00:00:00,Other,Unknown,MALE
40659574,2017-12-28 00:00:00,Other,28_to_32,MALE
7571525,2018-04-20 00:00:00,HCMC,33_to_37,MALE
28716109,2019-12-17 00:00:00,Other,33_to_37,FEMALE


Sucessfully load data to MSSQL SERVER

In [44]:
# Close the connect of pyodbc to MSSQL
cnxn.close()
# Dispose of the SQLAlchemy engine to close the connection
engine.dispose()